<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/deep%20learning/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Guia Prático de Fine-Tuning

Olá! Este notebook é seu guia passo a passo para o mundo do fine-tuning de LLMs. Como estudante de Engenharia de Computação, é crucial entender não apenas *como* fazer, mas *por que* as técnicas funcionam.



As Estratégias de Fine-Tuning

1.  **Full Fine-Tuning**:
    * **O que é?** Ajustar *todos* os bilhões de pesos do modelo.
    * **Problema:** Requer uma quantidade massiva de VRAM (memória de GPU), sendo inviável para modelos de 8B em hardware comum ou no Colab. Um modelo de 8B com precisão total (32-bit) precisaria de `8B * 4 bytes = 32 GB` de VRAM só para ser carregado, sem contar a memória extra para o treinamento.

2.  **PEFT (Parameter-Efficient Fine-Tuning)**:
    * **O que é?** Uma família de técnicas que congela os pesos originais do LLM (que são 99.9% do total) e treina apenas um número minúsculo de novos parâmetros "adaptadores".
    * **Vantagem:** Redução drástica no uso de memória e aceleração do treino.

3.  **LoRA (Low-Rank Adaptation)**:
    * **A Estrela do PEFT.** A ideia é que a "atualização" dos pesos para uma nova tarefa pode ser representada por matrizes de baixo posto (muito menores). Em vez de modificar uma matriz de peso gigante `W`, adicionamos o resultado de duas matrizes pequenas, `A` e `B`, que são treináveis (`W' = W + B*A`).

4.  **QLoRA (Quantized LoRA)**:
    * **A Magia que nos permite rodar no Colab.** É uma otimização do LoRA que faz duas coisas geniais:
        1.  **Quantização:** Carrega o modelo principal (Llama 3 8B) com precisão reduzida (4-bit em vez de 16-bit), cortando o uso de memória em 4x.
        2.  **Adaptação LoRA:** Treina os adaptadores LoRA normalmente sobre esse modelo quantizado.


## Parte 1

In [ ]:
!pip install unsloth

In [2]:
from huggingface_hub import login
from google.colab import userdata

# Retrieve the HF_TOKEN from Colab secrets and login
hf_token = userdata.get("HF_TOKEN")
login(token=hf_token)

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Download latest version
path = kagglehub.dataset_download("datasnaek/mbti-type")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mbti-type' dataset.
Path to dataset files: /kaggle/input/mbti-type


In [4]:
# Load the MBTI dataset from the local path
import pandas as pd
import os

# Assuming the dataset is downloaded to the default kagglehub path
# based on the output of the previous download cell.
# You might need to adjust this path if you downloaded it elsewhere.
dataset_path = "/root/.cache/kagglehub/datasets/datasnaek/mbti-type/versions/1/mbti_1.csv"

df = pd.read_csv(dataset_path)
print("Dataset loaded successfully from local path. First 5 records:")
display(df.head())

Dataset loaded successfully from local path. First 5 records:


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [5]:
from datasets import Dataset

# Preprocess the dataset
def format_mbti_data(examples):
    # Create a prompt that includes the MBTI type and the posts
    # The model will learn to generate text based on this format.
    # We'll use a simple format: "Personality Type: Posts"
    # You can experiment with different prompt formats.
    examples["text"] = [f"Personality Type: {mbti_type}\nPosts: {posts}" for mbti_type, posts in zip(examples["type"], examples["posts"])]
    return examples

# Apply the formatting function to the DataFrame to create a new column
df['text'] = df.apply(lambda row: f"Personality Type: {row['type']}\nPosts: {row['posts']}", axis=1)

# Convert the pandas DataFrame to a Hugging Face Dataset
# We'll only keep the 'text' column for fine-tuning
dataset = Dataset.from_pandas(df[['text']])

print("Formatted dataset example:")
print(dataset[0]['text'])
print("\nDataset structure:")
print(dataset)

Formatted dataset example:
Personality Type: INFJ
Posts: 'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.

In [6]:
# Use a smaller portion of the dataset for faster experimentation
# You can adjust the 'frac' parameter to change the percentage of data used.
# For example, frac=0.1 uses 10% of the dataset.
sample_size = 0.1 # Use 10% of the dataset
dataset = dataset.train_test_split(test_size=1-sample_size, seed=42)['train'] # Use a fixed seed for reproducibility

print(f"Using a sampled dataset with {len(dataset)} examples.")
print("Sampled dataset structure:")
print(dataset)

Using a sampled dataset with 867 examples.
Sampled dataset structure:
Dataset({
    features: ['text'],
    num_rows: 867
})


In [11]:
from unsloth import FastLanguageModel
import torch

# Carregando o modelo base usando unsloth
max_seq_length = 2048 # You can change this to the maximum length of your sequences
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Enables 4-bit quantization

# 1. Load the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-bnb-4bit", # Choose your model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use provided token to download weights like HF does
)

# 2. Apply LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number from 8 to 64
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # Recommended for Llama 3
    random_state = 3407,
    use_rslora = False,  # We support Rank Stabilized LoRA
    loftq_config = None, # And LoftQ
)

# No need to set pad_token or other configurations manually
# unsloth handles this automatically

# Print trainable parameters
model.print_trainable_parameters()

==((====))==  Unsloth 2025.9.11: Fast Gemma2 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.9.11 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


trainable params: 20,766,720 || all params: 2,635,108,608 || trainable%: 0.7881


In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Seus Argumentos do treinamento
training_args = TrainingArguments(
    output_dir="./unsloth/gemma-2b-json-finetune",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    max_steps=100,
    logging_steps=10,
    warmup_ratio=0.05,
    # bf16=True, # Unsloth handles this
    # optim="paged_adamw_8bit", # Unsloth handles this
    push_to_hub=False,
    report_to="wandb"
)

# 3. Passe o 'config' para o SFTTrainer e remova os argumentos antigos
trainer = SFTTrainer(
    model=model, # Use the unsloth model
    tokenizer=tokenizer, # Use the unsloth tokenizer
    train_dataset=dataset,
    args=training_args,
    dataset_text_field = "text", # Specify the text field in your dataset
    max_seq_length = max_seq_length,
)

# Inicia o treinamento
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/867 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 867 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 20,766,720 of 2,635,108,608 (0.79% trained)
wandb: Currently logged in as: isaac052503 (isaac052503-unicamp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,3.056000
20,2.982700
30,3.026400
40,2.983200
50,2.892900
60,2.875000
70,2.957800
80,2.900800
90,2.919400
100,2.938000


TrainOutput(global_step=100, training_loss=2.9532276725769044, metrics={'train_runtime': 491.964, 'train_samples_per_second': 0.813, 'train_steps_per_second': 0.203, 'total_flos': 5025460629178368.0, 'train_loss': 2.9532276725769044, 'epoch': 0.4608294930875576})

In [30]:
# Se em outro momento, voce quiser fazer o uso dos adapters, basta pegar o modelo base e
# juntar com os adapters


# # 1. Carregando o Modelo Base com unsloth para inferência
# base_model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/gemma-2-2b-bnb-4bit", # ou o modelo base que você usou
#     max_seq_length = 2048,
#     dtype = None,
#     load_in_4bit = True, # Carregue em 4 bits para inferência eficiente
# )

# # 2. Carregando o Adaptador PEFT e anexando ao modelo base
# adapter_path = "./unsloth/output/adapters" # Caminho onde você salvou os adaptadores
# peft_model = PeftModel.from_pretrained(base_model, adapter_path)

# # 3. Preparar o modelo PEFT para inferência
# FastLanguageModel.for_inference(peft_model)

# # Agora você pode usar 'peft_model' para inferência
# # ... (código para preparar inputs e gerar texto) ...

In [31]:
# nao eh necessario juntar os adapters aqui, porque o model acabou de ser treinado
# e atualizado com as novas informacoes, se quiser usar as informacoes novas em outra
# ocasiao sem treinar, deve fazer o procedimento acima

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# Use the Gemma prompt format
test_instruction = "Uma dúvida, se eu sou mais animado, sou INTJ?"
test_input = ""
gemma_prompt = f"<start_of_turn>user\n{test_instruction}\n{test_input}<end_of_turn>\n<start_of_turn>model\n"

inputs = tokenizer(
    [gemma_prompt], return_tensors = "pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 30, use_cache = False)
print(tokenizer.batch_decode(outputs))

['<bos><start_of_turn>user\nUma dúvida, se eu sou mais animado, sou INTJ?\n<end_of_turn>\n<start_of_turn>model\nUma dúvida, se eu sou mais animado, sou INTJ?\n帖最后由 user12345678901234']


In [19]:
model.save_pretrained("./unsloth/output/adapters")  # Local saving
tokenizer.save_pretrained("./unsloth/output/adapters")


# If you can load the adapter when using the model base now

('./unsloth/output/adapters/tokenizer_config.json',
 './unsloth/output/adapters/special_tokens_map.json',
 './unsloth/output/adapters/tokenizer.model',
 './unsloth/output/adapters/added_tokens.json',
 './unsloth/output/adapters/tokenizer.json')

## Saiba Mais


Parabéns, você treinou um adaptador LoRA! No entanto, no estado atual, você tem duas partes: o modelo base e seu pequeno adaptador LoRA. Para fazer uma inferência, você precisa carregar ambos.

A mesclagem é o processo de "fundir" os pesos do seu adaptador LoRA de volta aos pesos do modelo base. O resultado é um único modelo autônomo que já contém a sua especialização.

Mas e se você tivesse treinado **vários adaptadores** LoRA?
* Um para aprender sobre personalidades.
* Outro para ser um especialista em programação Python.
* Um terceiro para escrever de forma criativa.

Você pode mesclá-los para criar um "super-modelo" que faz tudo isso. A função `.merge_and_unload()` não é ideal para isso. A ferramenta padrão da comunidade é a **`mergekit`**.

`mergekit` funciona com um arquivo de configuração YAML onde você especifica os modelos e a estratégia de mesclagem.
